### Generator of Midi songs

Importing important libraries

In [5]:
import pickle
import numpy
from music21 import instrument, note, stream, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation


from generation_utils import prepare_sequences, generate_notes
from midi_creator import create_midi
import warnings
warnings.simplefilter("ignore")

Implementing main LSTM model, that load trained weights

In [4]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
#     model.add(LSTM(
#         1024,
#         input_shape=(input.shape[1], input.shape[2]),
#         return_sequences=True
#     ))
#     model.add(Dropout(0.3))
#     model.add(LSTM(1024, return_sequences=True)) -- Advanced model, more parameters, not commented - default model
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    model.load_weights('weights/best_weights.hdf5')
#     load them if you need trained weights
#     model.load_weights('weights/worst_weight.hdf5')
#     model.load_weights('weights/middle_weights.hdf5')    
#     model.load_weights('weights/middle2_weights.hdf5.hdf5')
    return model

Reading all chords and notes from file and create our vocab_size and all unique pitches

In [6]:
with open('data/notes', 'rb') as filepath:
    notes = pickle.load(filepath)
    
pitches = sorted(set(item for item in notes))
vocab_size = len(set(notes))

Creating network input and normalized network input. First one used in generating new songs, second one used for passing parameters through LSTM model

In [7]:
network_input, normalized_input = prepare_sequences(notes, pitches, vocab_size)
LSTM = create_network(normalized_input, vocab_size)

Here our default model

In [8]:
LSTM.summary() # DEFAULT MODEL

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 512)          1052672   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
__________

#### Generation of midi songs process

Predicted next chords/notes in sequence with length = 100

In [9]:
prediction_output = generate_notes(LSTM, network_input, pitches, vocab_size)

In [12]:
len(prediction_output)

100

Run create_midi function for generating midi songs from predicted output of LSTM

In [14]:
create_midi(prediction_output)